In [192]:
import glob
import os
import pandas as pd
import seaborn as sns
import re
import numpy as np


df = pd.read_csv ('forms.csv')          # 
hausa = df[(df.Language_ID == 4)]


ModuleNotFoundError: No module named 'nltk'

In [193]:
hausa = (hausa.loc[(hausa['BorrowedScore'] == 0) & (hausa['Analyzability'] == "unanalyzable")])
reduced_words = hausa[~hausa['Form'].str.contains(r'\d|,|`|\'|\$|\ \(|\)' + '|'.join(cplx_onset), regex=True)].drop_duplicates(subset = 'Form')
reduced_words.index = np.arange(len(reduced_words))
len(reduced_words)
reduced_words

,ID,Language_ID,Parameter_ID,Form,Segments,Comment,Source,Contribution_ID,Word_ID,BorrowedScore,...,lexical_stratum,loan_history,numeric_frequency,original_script,other_comments,reference,register,relative_frequency,salience,word_source
0,10667,4,1-21,ƙásáa,NaN,NaN,4,4,73101839578931380,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10669,4,1-213,ƙùuráa,NaN,NaN,4,4,73101839326746903,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10670,4,1-214,tàɓóo,NaN,NaN,4,4,73101839376009506,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10673,4,1-22,tsáunìi,NaN,NaN,4,4,7310183922231862,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10674,4,1-22,dúutsèe,NaN,NaN,4,4,73101839531361252,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,10675,4,1-22,túdùu,NaN,NaN,4,4,73101839520738405,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,10676,4,1-222,háyìi,NaN,NaN,4,4,73101839297141366,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,10679,4,1-24,ràafíi,NaN,NaN,4,4,73101839665354033,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,10680,4,1-25,tsìbíríi,NaN,NaN,4,4,73101839325131461,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10682,4,1-27,gáaɓàa,NaN,NaN,4,4,73101839481491358,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
pd.options.display.max_rows = 999

In [ ]:
reduced_words = reduced_words['Form']

In [194]:
cplx_onset = ['ts','gw','ƙw','ƙy','sh','kw']
coda = 'sfr̃mnrzl'


In [ ]:
class Autorep:
    def __init__(self, word):
        self.word = word
        self.tone = ''
        self.mldy = ''
        self.assoc = {}

        # get the tone
        for seg in word:
            

In [217]:
h_tone = "áéíóú"
l_tone = "àèìòù"
f_tone = "âêîôû"
long_vowel = 'aeiou'
v = h_tone+l_tone+f_tone

words = ['ƙásáa','ƙámsáa','ƙásá','ƙássâs']
for word in words:
    tone = ''  # Initialize tone outside the loop
    i = 0
    j = 1
    while j < len(word):  # Adjust loop condition to avoid index out of range
        if i < len(word):
            if word[i] in h_tone:
                if word[j] in long_vowel:
                    tone += 'H:'
                else:
                    tone += 'H'
            elif word[i] in l_tone:
                if word[j] in long_vowel:
                    tone += 'L:'
                else:
                    tone += 'L'
            elif word[i] in f_tone:
                tone += 'F'
            

        if i == len(word):
            if word[i] in h_tone:
                tone += 'H'
            elif word[i] in l_tone:
                tone += 'L'    
        i += 1
        j+= 1        
    print(f"Word: {word}, Tone: {tone}")   
   

Word: ƙásáa, Tone: HH:
Word: ƙámsáa, Tone: HH:
Word: ƙásá, Tone: H
Word: ƙássâs, Tone: HF


In [199]:

words = ['ƙásáa','ƙámsáa','ƙásá','ƙássâs']
len(words[0])
words[0][4]

'a'